##* 종목 뉴스 분류에 쓸 Dataset 만들기
- 종목: cluster_8 '034950', cluster_7 - '005930', cluster_2 - '328380'
  - 종목별 분별 데이터 셋

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
!pip install JPype1
#!pip install konlpy
!pip install pyMySQL
#!pip install -U finance-datareader

In [27]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [28]:
#-- make_bun_sokbo_dataset()----------------------------------
# 종목별 분 기준 데이터셋 만들기
def make_bun_sokbo_dataset(stock_code):
  # 시작일
  sdate='20230101' 
  # 종료일
  edate='20240112' 
  # 현재, 초기화
  ddate= sdate     
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 데이터프레임 생성
  all_set_df=pd.DataFrame()
  # 데이터프레임 생성
  oneday_set_df=pd.DataFrame()
  # 시작일부터 종료일까지 처리
  while pd.to_datetime(ddate)<=pd.to_datetime(edate):
   # DB에서 종목 데이터  추출위한 sql 준비
    sql = f"SELECT x.2000_encoded_text_code, x.{stock_code}_bun_label FROM {ddate}_sokbo_flatten_label x"
    # DB 검색결과를 dataframe에 저장
    oneday_set_df = pd.read_sql_query(sql, conn)
    # 실행확인을 위한 화면 출력
    print(ddate)
    # 공백 행 삭제
    oneday_set_df = oneday_set_df[oneday_set_df['2000_encoded_text_code'].str.len() > 0]
    # 실행확인을 위한 화면 출력
    print(len(oneday_set_df))
    # 당일 데이터 붙이기
    all_set_df = pd.concat([all_set_df,oneday_set_df],ignore_index=True)
    # 실행확인을 위한 화면 출력
    print(len(all_set_df))
    # 날짜를 하루 증가시키기
    ddate = pd.to_datetime(ddate)+pd.Timedelta(days=1)
    # YYYYMMDD 형식의 문자열로 변환
    ddate = ddate.strftime('%Y%m%d')
  # DB close
  conn.close()
  # 실행확인을 위한 화면 출력
  print(len(all_set_df))
  #------------------------------------------------------------------------
  # DB에 데이터 셋 저장
  # DB 저장 연결 준비
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  # DB 커넥션 생성
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
  all_set_df.to_sql(name=stock_code+'_bun_dataset', con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()
  #-------------------------------------------------------------------------

#-- make_bun_sokbo_dataset(): End ------------------------------------------

In [ ]:
#============================================================
# 메인 함수 호출: 분기준 데이터 셋 만들기, 예)종목 '034950'
#------------------------------------------------------------
make_bun_sokbo_dataset('034950')
#------------------------------------------------------------

In [ ]:
#============================================================
# 메인 함수 호출: 일기준 데이터 셋 만들기, 예)종목 '005930'
#------------------------------------------------------------
make_bun_sokbo_dataset('005930')
#============================================================

In [ ]:
#============================================================
# 메인 함수 호출: 일기준 데이터 셋 만들기, 예)종목 '328380'
#------------------------------------------------------------
make_bun_sokbo_dataset('328380')
#============================================================